In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [ ]:
SEQ_LENGTH = 20
DATA_DIR = "eyeblink8" 
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)

def calculate_ear(landmarks, eye_indices):
    p = [landmarks[i] for i in eye_indices]
    v1 = np.linalg.norm(np.array([p[1].x, p[1].y]) - np.array([p[5].x, p[5].y]))
    v2 = np.linalg.norm(np.array([p[2].x, p[2].y]) - np.array([p[4].x, p[4].y]))
    h = np.linalg.norm(np.array([p[0].x, p[0].y]) - np.array([p[3].x, p[3].y]))
    return (v1 + v2) / (2.0 * h)

def load_labels(tag_file):
    labels = []
    with open(tag_file, 'r') as f:
        lines = f.readlines()
        start_idx = 0
        for i, line in enumerate(lines):
            if "#start" in line:
                start_idx = i + 1
                break
        for line in lines[start_idx:]:
            parts = line.split(':')
            if len(parts) > 1:
                # 1 jika sedang berkedip, 0 jika tidak
                is_blinking = 1 if parts[2].strip() != 'X' else 0
                labels.append(is_blinking)
    return labels

In [ ]:
X_all, y_all = [], []

folders = [f for f in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, f))]

print(f"Memproses {len(folders)} folder video...")

for folder in folders:
    folder_path = os.path.join(DATA_DIR, folder)
    video_files = [f for f in os.listdir(folder_path) if f.endswith('.avi')]
    tag_files = [f for f in os.listdir(folder_path) if f.endswith('.tag')]
    
    if not video_files or not tag_files: continue
    
    labels = load_labels(os.path.join(folder_path, tag_files[0]))
    cap = cv2.VideoCapture(os.path.join(folder_path, video_files[0]))
    
    video_ears = []
    frame_idx = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or frame_idx >= len(labels): break
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_frame)
        
        if results.multi_face_landmarks:
            landmarks = results.multi_face_landmarks[0].landmark
            ear = (calculate_ear(landmarks, LEFT_EYE) + calculate_ear(landmarks, RIGHT_EYE)) / 2
            video_ears.append(ear)
        else:
            video_ears.append(video_ears[-1] if video_ears else 0.3)
        frame_idx += 1
    cap.release()

    # Buat sequence per video agar tidak ada kebocoran antar subjek
    for i in range(len(video_ears) - SEQ_LENGTH):
        X_all.append(video_ears[i:i + SEQ_LENGTH])
        # LABEL PERBAIKAN: Gunakan status frame TERAKHIR dalam sequence
        y_all.append(labels[i + SEQ_LENGTH - 1])

X_all = np.array(X_all).reshape(-1, SEQ_LENGTH, 1)
y_all = np.array(y_all)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=42)

In [ ]:
model = tf.keras.Sequential([
    layers.Input(shape=(SEQ_LENGTH, 1)),
    layers.LSTM(32, return_sequences=False), # Layer dikurangi agar tidak overfit
    layers.Dropout(0.3),                      # Dropout ditingkatkan
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    X_train, y_train, 
    epochs=20, 
    batch_size=32, 
    validation_data=(X_test, y_test)
)

In [ ]:
model.save("blink_detection_model.h5")
model.save("blink_detection_model.keras")